# SCARF - A Low-Latency Block Cipher for Secure Cache-Randomization

### Prerequisites

In [1]:
import random

### Utility bitwise functions

In [2]:
def rotate_left(x, bits, max_bits):
    return ((x << bits % max_bits) & ((1 << max_bits) - 1)) | (x >> (max_bits - (bits % max_bits)))

def rotate_right(x, bits, max_bits):
    return ((x >> (bits % max_bits)) | (x << (max_bits - (bits % max_bits)))) & ((1 << max_bits) - 1)

def NOT(x, bits):
    return x ^ ((1 << bits) - 1)

In [3]:
assert rotate_left(0b10011, 2, 5) == 0b01110, "rotate_left failed"
assert rotate_right(0b01110, 2, 5) == 0b10011, "rotate_right failed"
assert NOT(0b10011, 5) == 0b01100, "NOT failed"

### S and G functions

In [4]:
def generate_S(x):
    return ((rotate_left(x, 0, 5) | rotate_left(x, 1, 5)) & (NOT(rotate_left(x, 3, 5), 5) | NOT(rotate_left(x, 4, 5), 5))) ^ \
           ((rotate_left(x, 0, 5) | rotate_left(x, 2, 5)) & (NOT(rotate_left(x, 2, 5), 5) | rotate_left(x, 3, 5)))

# Precompute S-box and its inverse, max value is 2^5 - 1 (5 bits)
SBOX = [generate_S(x) for x in range(32)]
INVERSE_SBOX = [0] * 32

for i, s in enumerate(SBOX):
    INVERSE_SBOX[s] = i

def S(x):
    return SBOX[x]

def S_inverse(x):
    return INVERSE_SBOX[x]

def G(x, k):
    result = 0
    for i in range(5):
        result ^= rotate_left(x, i, 5) & k[i]

    result = result ^ rotate_left(x, 1, 5) & rotate_left(x, 2, 5)
    return result

In [5]:
assert S_inverse(S(0b10011)) == 0b10011, "S_inverse failed"
assert all(S_inverse(S(x)) == x for x in range(32)), "S not invertible for all values"

### Round functions and inverses

In [6]:
def R1(x, key):
    # Divide key into 6 parts of 5 bits each
    k = [(key >> (i * 5)) & 0x1F for i in range(6)]

    # Get upper (x_left) and lower (x_right) 5 bits of x
    x_left = (x >> 5) & 0x1F
    x_right = x & 0x1F

    y = G(x_left, k) ^ x_right
    x_right = S(x_left ^ k[5])

    # Concatenate the left and right halves
    return (y << 5) | x_right

def R1_inv(x, key):
    # Divide key into 6 parts of 5 bits each
    k = [(key >> (i * 5)) & 0x1F for i in range(6)]

    # Get upper (x_left) and lower (x_right) 5 bits of x
    x_left = (x >> 5) & 0x1F
    x_right = x & 0x1F

    # First recover x_left
    x_left = S_inverse(x_right) ^ k[5]

    # Then recover x_right using G
    y = x >> 5  # Extract y from the input
    x_right = y ^ G(x_left, k)

    # Concatenate the left and right halves
    return (x_left << 5) | x_right

def R2(x, key):
    # Divide key into 6 parts of 5 bits each
    k = [(key >> (i * 5)) & 0x1F for i in range(6)]

    # Get upper (x_left) and lower (x_right) 5 bits of x
    x_left = (x >> 5) & 0x1F
    x_right = x & 0x1F

    # Compute the new left and right halves
    x_right = G(x_left, k) ^ x_right
    x_left = S(x_left) ^ k[5]

    # Concatenate the left and right halves
    return (x_left << 5) | x_right

def R2_inv(x, key):
    k = [(key >> (i * 5)) & 0x1F for i in range(6)]
    x_left = (x >> 5) & 0x1F
    x_right = x & 0x1F

    # First recover x_left
    x_left = S_inverse(x_left ^ k[5])
    
    # Then recover x_right using G
    x_right = x_right ^ G(x_left, k)

    return (x_left << 5) | x_right

In [7]:
round_key = random.getrandbits(30)
x_test = random.getrandbits(10)

assert R1_inv(R1(x_test, round_key), round_key) == x_test, "R1_inv failed"
assert R2_inv(R2(x_test, round_key), round_key) == x_test, "R2_inv failed"
assert R1(R1_inv(x_test, round_key), round_key) == x_test, "R1_inv failed"
assert R2(R2_inv(x_test, round_key), round_key) == x_test, "R2_inv failed"

### Expansion function


In [8]:
def expansion(tweak):
    # Expand the 48-bit tweak into a 60-bit value by adding a 0 bit between each 4 bits
    
    result = 0
    for i in range(12):
        nibble = (tweak >> (i * 4)) & 0xF
        result |= nibble << (i * 5)

    return result

In [9]:
test_tweak = (1 << 48) - 1
expanded_test_tweak = expansion(test_tweak)

print(f"Test Tweak:     {test_tweak:048b}")
print(f"Expanded Tweak: {expanded_test_tweak:060b}")

Test Tweak:     111111111111111111111111111111111111111111111111
Expanded Tweak: 011110111101111011110111101111011110111101111011110111101111


### SL, Sigma, pi functions

In [10]:
def SL(x):
    # The S-box from the round functions is applied to each 5-bit chunk of the input
    # and the results are concatenated.

    result = 0
    for i in reversed(range(12)):
        result <<= 5
        result |= S((x >> (i * 5)) & 0x1F)

    return result

def Sigma(x):
    return x ^ rotate_left(x, 6, 60) ^ rotate_left(x, 12, 60) ^ rotate_left(x, 19, 60) ^ rotate_left(x, 29, 60) ^ rotate_left(x, 43, 60) ^ rotate_left(x, 51, 60)

# Generate the permutation table
p = [j * 5 + i for i in range(5) for j in range(12)]

def pi(x):
    result = 0
    for i in range(60):
        bit = (x >> i) & 1
        result |= bit << p[i]

    return result

### Tweakey scheduling, round key generator functions

In [11]:
def tweakey_schedule(key, tweak):
    tk = [0] * 4

    # Split 240-bit key into 4 parts of 60 bits each
    k = [key >> (i * 60) & ((1 << 60) - 1) for i in range(4)]

    # Compute subkeys
    tk[0] = expansion(tweak) ^ k[0]
    tk[1] = Sigma(SL(tk[0])) ^ k[1]
    tk[2] = SL(pi(SL(tk[1]) ^ k[2]))
    tk[3] = SL(Sigma(tk[2]) ^ k[3])

    return tk

def generate_round_keys(tk):
    round_keys = [0] * 8
    
    for i in range(4):
        round_keys[2 * i] = tk[i] & ((1 << 30) - 1)
        round_keys[2 * i + 1] = (tk[i] >> 30) & ((1 << 30) - 1)

    return round_keys

In [12]:
test_key = random.getrandbits(240)
test_tweak = random.getrandbits(48)
test_tk = tweakey_schedule(test_key, test_tweak)
test_round_keys = generate_round_keys(test_tk)

assert (test_round_keys[1] << 30) | test_round_keys[0] == test_tk[0], "Key schedule failed"
assert (test_round_keys[3] << 30) | test_round_keys[2] == test_tk[1], "Key schedule failed"
assert (test_round_keys[5] << 30) | test_round_keys[4] == test_tk[2], "Key schedule failed"
assert (test_round_keys[7] << 30) | test_round_keys[6] == test_tk[3], "Key schedule failed"

### Encryption, decryption functions

In [13]:
def encrypt(plaintext, round_keys):
    # Start with plaintext
    state = plaintext

    # Apply 7 R1 rounds
    for i in range(7):
        state = R1(state, round_keys[i])

    # Apply 1 R2 round
    state = R2(state, round_keys[7])

    return state

def decrypt(ciphertext, round_keys):
    # Start with ciphertext
    state = ciphertext

    # Apply 1 R2_inv round
    state = R2_inv(state, round_keys[7])

    # Apply 7 R1_inv rounds in reverse order
    for i in range(6, -1, -1):
        state = R1_inv(state, round_keys[i])

    return state


key = random.getrandbits(240)
tweak = random.getrandbits(48)
plaintext = random.getrandbits(10)

print(f"Key: {key:240b}")
print(f"Tweak: {tweak:048b}")

tk = tweakey_schedule(key, tweak)
round_keys = generate_round_keys(tk)

for i in range(4):
    print(f"Tweakey {i}: {tk[i]:060b}")

for i in range(8):
    print(f"Round Key {i}: {round_keys[i]:030b}")

ciphertext = encrypt(plaintext, round_keys)
decrypted_plaintext = decrypt(ciphertext, round_keys)

print(f"Plaintext:  {plaintext:010b}")
print(f"Ciphertext: {ciphertext:010b}")
print(f"Decrypted:  {decrypted_plaintext:010b}")

Key: 100001110011000010110001011100001000001011101110000101111010100110001000101011010110101011100100110000010000001010101001110001001011010010111101110010000000100001001110000100011101111101001000100100111001010101011011110000100100101100100100
Tweak: 001110011011001110000110011100000101110111111100
Tweakey 0: 111011101101111100001101010011010101110000001111111011001000
Tweakey 1: 110010100101110010101100000001010101111111100101001011100101
Tweakey 2: 000110110101010111010110010010001001000101000111110010111110
Tweakey 3: 010011001001110111000000100100101100001101110111111001101000
Round Key 0: 010101110000001111111011001000
Round Key 1: 111011101101111100001101010011
Round Key 2: 010101111111100101001011100101
Round Key 3: 110010100101110010101100000001
Round Key 4: 001001000101000111110010111110
Round Key 5: 000110110101010111010110010010
Round Key 6: 101100001101110111111001101000
Round Key 7: 010011001001110111000000100100
Plaintext:  1010001001
Ciphertext: 1001110100
Decrypted:  

In [14]:
assert 0 <= plaintext < (1 << 10), "Plaintext must be 10 bits"
assert 0 <= tweak < (1 << 48), "Tweak must be 48 bits"
assert 0 <= key < (1 << 240), "Key must be 240 bits"
assert 0 <= ciphertext < (1 << 10), "Ciphertext must be 10 bits"
assert 0 <= decrypted_plaintext < (1 << 10), "Decrypted plaintext must be 10 bits"
assert decrypted_plaintext == plaintext, "Decryption failed"